In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
from geopy.geocoders import Nominatim
from urllib.request import urlopen
import json

In [2]:
print("Pandas Version: {}".format(pd.__version__))
print("NumPy Version: {}".format(np.__version__))
!python --version
!nvidia-smi

Pandas Version: 1.3.5
NumPy Version: 1.21.1
Python 3.8.10
Sun Oct 15 20:44:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   35C    P0    39W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                              

In [3]:
data_path = "../data/bridges.csv"
data_types = {'#': str,
              'Name': str,
              'Year': str,
              'Location': str,
              'Country': str,
              'Status': str
             }

bridges_df = pd.read_csv(data_path, dtype = data_types)

In [4]:
bridges_df

,#,Name,Year,Location,Country,Status,Type
0,1,470 Odra River Bridge,2009,Ostrava,Czechia,in use,composite
1,2,4708/C203 Bridge,2009,Ostrava,Czechia,in use,composite
2,3,7th Avenue Pedestrian Bridge (N1),2010,Johannesburg,South Africa,in use,composite
3,4,A 1 Lenne Bridge,2009,Hagen (NRW),Germany,in use,composite
4,5,A 113 Teltowkanal Bridge,2004,- Berlin (BE),Germany,in use,composite
...,...,...,...,...,...,...,...
15478,7953,Zwingenberg Bridge,2011,Zwingenberg (BW),Germany,in use,metal
15479,7954,Zwischenbrunner Strasse Bridge,NaN,Sankt Pölten (NOE),Austria,in use,metal
15480,7955,Zwolle Railroad Bridge,2011,Zwolle,Netherlands,in use,metal
15481,7956,Żywiec Bridge,NaN,Zywiec (SL),Poland,in use,metal


In [5]:
bridges_df_no_na_year = bridges_df.dropna(subset = ['Year']).reset_index(drop = True)
bridges_df_no_na_year

,#,Name,Year,Location,Country,Status,Type
0,1,470 Odra River Bridge,2009,Ostrava,Czechia,in use,composite
1,2,4708/C203 Bridge,2009,Ostrava,Czechia,in use,composite
2,3,7th Avenue Pedestrian Bridge (N1),2010,Johannesburg,South Africa,in use,composite
3,4,A 1 Lenne Bridge,2009,Hagen (NRW),Germany,in use,composite
4,5,A 113 Teltowkanal Bridge,2004,- Berlin (BE),Germany,in use,composite
...,...,...,...,...,...,...,...
11873,7950,Zweigert Bridge,2000,Essen (NRW),Germany,in use,metal
11874,7952,Zwierzyniecki Bridge,1897,Wroclaw (DS),Poland,in use,metal
11875,7953,Zwingenberg Bridge,2011,Zwingenberg (BW),Germany,in use,metal
11876,7955,Zwolle Railroad Bridge,2011,Zwolle,Netherlands,in use,metal


In [6]:
bridges = bridges_df_no_na_year.drop(columns = ['#'])
bridges

,Name,Year,Location,Country,Status,Type
0,470 Odra River Bridge,2009,Ostrava,Czechia,in use,composite
1,4708/C203 Bridge,2009,Ostrava,Czechia,in use,composite
2,7th Avenue Pedestrian Bridge (N1),2010,Johannesburg,South Africa,in use,composite
3,A 1 Lenne Bridge,2009,Hagen (NRW),Germany,in use,composite
4,A 113 Teltowkanal Bridge,2004,- Berlin (BE),Germany,in use,composite
...,...,...,...,...,...,...
11873,Zweigert Bridge,2000,Essen (NRW),Germany,in use,metal
11874,Zwierzyniecki Bridge,1897,Wroclaw (DS),Poland,in use,metal
11875,Zwingenberg Bridge,2011,Zwingenberg (BW),Germany,in use,metal
11876,Zwolle Railroad Bridge,2011,Zwolle,Netherlands,in use,metal


In [7]:
dict(bridges['Country'].value_counts())

{'Germany': 2434,
 'USA': 1974,
 'France': 1228,
 'Italy': 564,
 'Spain': 556,
 'United Kingdom': 485,
 'Switzerland': 406,
 'Japan': 366,
 'China': 334,
 'Netherlands': 250,
 'Austria': 201,
 'Canada': 186,
 'Belgium': 171,
 'Russia': 156,
 'Poland': 135,
 'Rwanda': 126,
 'Norway': 119,
 'Czechia': 107,
 'Australia': 103,
 'New Zealand': 96,
 'South Korea': 87,
 'Portugal': 83,
 'Sweden': 78,
 'India': 76,
 'Finland': 66,
 'Bolivia': 62,
 'Nicaragua': 61,
 'Luxembourg': 57,
 'Denmark': 48,
 'Brazil': 47,
 'Taiwan': 46,
 'Ukraine': 44,
 'Ireland': 41,
 'Panama': 36,
 'Mexico': 36,
 'South Africa': 35,
 'Guatemala': 33,
 'Réunion': 32,
 'Serbia': 31,
 'Iran': 30,
 'Uganda': 27,
 'Hungary': 27,
 'Greece': 26,
 'Croatia': 25,
 'Slovenia': 23,
 'Colombia': 23,
 'Turkey': 21,
 'Romania': 20,
 'Vietnam': 20,
 'Argentina': 20,
 'Indonesia': 18,
 'Israel': 18,
 'Morocco': 18,
 'Slovakia': 18,
 'Peru': 18,
 'Ethiopia': 17,
 'Malaysia': 17,
 'Bangladesh': 16,
 'Venezuela': 16,
 'Thailand': 15,
 

In [8]:
#na_bridges = bridges[(bridges['Country'] == "USA") | (bridges['Country'] == "Canada")].reset_index(drop = True)
us_bridges = bridges[bridges['Country'] == "USA"].reset_index(drop = True)

In [9]:
us_bridges

,Name,Year,Location,Country,Status,Type
0,Abernethy Bridge,1970,Oregon City (OR),USA,in use,composite
1,Allegheny Station Viaduct,2012,Pittsburgh (PA),USA,in use,composite
2,Antioch Bridge,1978,- Antioch (CA),USA,in use,composite
3,Bagley Street Pedestrian Bridge,2010,Detroit (MI),USA,in use,composite
4,Bear Creek Bridge,2011,Ouray County (CO),USA,in use,composite
...,...,...,...,...,...,...
1969,Yellowstone River Bridge,1930,Gardiner (MT),USA,in use,metal
1970,Yuma Crossing Railroad Bridge,1923,Yuma (AZ),USA,in use,metal
1971,Zablocki Drive Bridge,1960,Milwaukee (WI),USA,in use,metal
1972,Zieglers Ford Bridge,1904,Decoria Township (MN),USA,demolished,metal


In [10]:
raw_cities = us_bridges['Location'].str.split('(').str[0]
raw_states = us_bridges['Location'].str.split('(').str[1]

In [11]:
cities = raw_cities.str[:-1].replace(r'^[^a-zA-Z]+', '', regex = True)

In [12]:
states = raw_states.str[:-1]

In [13]:
cities.value_counts()

New York       56
Chicago        53
Pittsburgh     38
Los Angeles    29
Minneapolis    23
               ..
Revere          1
Crestwood       1
Barton          1
Pe Ell          1
Zoarville       1
Name: Location, Length: 967, dtype: int64

In [14]:
states.value_counts()

CA       181
NY       162
PA       142
WA       137
OR       135
IL       117
OH        90
MN        67
TX        67
MA        53
NJ        53
NE        49
CT        38
MO        36
AZ        35
LA        35
WV        34
FL        34
WI        30
KS        30
KY        29
MI        28
TN        27
MD        26
CO        21
IA        21
VA        21
AL        19
GA        17
DE        17
WY        15
AK        14
ME        14
IN        14
MS        14
ID        13
MT        12
RI        12
AR        12
OK        11
UT        11
SC         9
NV         8
VT         7
NH         6
NC         6
NM         5
US-PR      5
HI         5
ND         4
SD         4
AS         1
Name: Location, dtype: int64

In [15]:
us_bridges['city'] = cities
us_bridges['state'] = states

In [16]:
us_bridges

,Name,Year,Location,Country,Status,Type,city,state
0,Abernethy Bridge,1970,Oregon City (OR),USA,in use,composite,Oregon City,OR
1,Allegheny Station Viaduct,2012,Pittsburgh (PA),USA,in use,composite,Pittsburgh,PA
2,Antioch Bridge,1978,- Antioch (CA),USA,in use,composite,Antioch,CA
3,Bagley Street Pedestrian Bridge,2010,Detroit (MI),USA,in use,composite,Detroit,MI
4,Bear Creek Bridge,2011,Ouray County (CO),USA,in use,composite,Ouray County,CO
...,...,...,...,...,...,...,...,...
1969,Yellowstone River Bridge,1930,Gardiner (MT),USA,in use,metal,Gardiner,MT
1970,Yuma Crossing Railroad Bridge,1923,Yuma (AZ),USA,in use,metal,Yuma,AZ
1971,Zablocki Drive Bridge,1960,Milwaukee (WI),USA,in use,metal,Milwaukee,WI
1972,Zieglers Ford Bridge,1904,Decoria Township (MN),USA,demolished,metal,Decoria Township,MN


In [17]:
us_bridges.isna().sum()

Name         0
Year         0
Location     2
Country      0
Status       1
Type         0
city         2
state       21
dtype: int64

In [18]:
us_bridges[us_bridges['state'].isna()]

,Name,Year,Location,Country,Status,Type,city,state
82,Arlington Memorial Bridge,1932,Washington,USA,in use,concrete,Washingto,NaN
241,Francis Scott Key Bridge,1923,Washington,USA,in use,concrete,Washingto,NaN
398,New Harvard Street Bridge,1965,Washington,USA,in use,concrete,Washingto,NaN
412,Old Harvard Street Bridge,1901,Washington,USA,in use,concrete,Washingto,NaN
433,Pennsylvania Avenue Bridge,1916,Washington,USA,in use,concrete,Washingto,NaN
579,William Howard Taft Bridge,1907,Washington,USA,in use,concrete,Washingto,NaN
642,Aqueduct Bridge,1888,Washington,USA,demolished,metal,Washingto,NaN
646,Arland D. Williams Jr. Memorial Bridge,1950,Washington,USA,in use,metal,Washingto,NaN
647,Arlington Memorial Bridge,1932,Washington,USA,in use,metal,Washingto,NaN
830,Chain Bridge,1938,Washington,USA,in use,metal,Washingto,NaN


In [19]:
us_bridges['state'] = us_bridges[['Location', 'state']].apply(lambda r: 'WA' if r.Location == 'Washington' else r.state, axis = 1)
us_bridges.drop(us_bridges[us_bridges.Location == 'Gulf of Mexico'].index, inplace=True)
us_bridges['city'] = us_bridges.apply(lambda r: 'Memphis' if r.Name == 'Frisco Bridge' else 'New Orleans' if r.Name == 'Huey P. Long Bridge' else r.city, axis = 1)
us_bridges['state'] = us_bridges.apply(lambda r: 'TN' if r.Name == 'Frisco Bridge' else 'LA' if r.Name == 'Huey P. Long Bridge' else r.state, axis = 1)
us_bridges[us_bridges['state'].isna()]

,Name,Year,Location,Country,Status,Type,city,state


In [20]:
state_names = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AS": "American Samoa",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "DC": "District Of Columbia",
    "FM": "Federated States Of Micronesia",
    "FL": "Florida",
    "GA": "Georgia",
    "GU": "Guam",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MH": "Marshall Islands",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "MP": "Northern Mariana Islands",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PW": "Palau",
    "PA": "Pennsylvania",
    "PR": "Puerto Rico",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VI": "Virgin Islands",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming",
    "US-PR": "Puerto Rico"
}

In [21]:
us_bridges['state_names'] = us_bridges['state'].replace(state_names)

In [22]:
us_bridges

,Name,Year,Location,Country,Status,Type,city,state,state_names
0,Abernethy Bridge,1970,Oregon City (OR),USA,in use,composite,Oregon City,OR,Oregon
1,Allegheny Station Viaduct,2012,Pittsburgh (PA),USA,in use,composite,Pittsburgh,PA,Pennsylvania
2,Antioch Bridge,1978,- Antioch (CA),USA,in use,composite,Antioch,CA,California
3,Bagley Street Pedestrian Bridge,2010,Detroit (MI),USA,in use,composite,Detroit,MI,Michigan
4,Bear Creek Bridge,2011,Ouray County (CO),USA,in use,composite,Ouray County,CO,Colorado
...,...,...,...,...,...,...,...,...,...
1969,Yellowstone River Bridge,1930,Gardiner (MT),USA,in use,metal,Gardiner,MT,Montana
1970,Yuma Crossing Railroad Bridge,1923,Yuma (AZ),USA,in use,metal,Yuma,AZ,Arizona
1971,Zablocki Drive Bridge,1960,Milwaukee (WI),USA,in use,metal,Milwaukee,WI,Wisconsin
1972,Zieglers Ford Bridge,1904,Decoria Township (MN),USA,demolished,metal,Decoria Township,MN,Minnesota


In [23]:
#sources:
#cont. US + HI + AK: https://www.currentresults.com/Weather/US/average-annual-state-temperatures.php
#AS + PR: https://www.weather.gov/wrh/Climate
state_avg_temps = {
    'Alabama': 62.8,
    'Alaska': 26.6,
    'Arizona': 60.3,
    'Arkansas': 60.4,
    'California': 59.4,
    'Colorado': 45.1,
    'Connecticut': 49.0,
    'Delaware': 55.3,
    'Florida': 70.7,
    'Georgia': 63.5,
    'Hawaii': 70.0,
    'Idaho': 44.4,
    'Illinois': 51.8,
    'Indiana': 51.7,
    'Iowa': 47.8,
    'Kansas': 54.3,
    'Kentucky': 55.6,
    'Louisiana': 66.4,
    'Maine': 41.0,
    'Maryland': 54.2,
    'Massachusetts': 47.9,
    'Michigan': 44.4,
    'Minnesota': 41.2,
    'Mississippi': 63.4,
    'Missouri': 54.5,
    'Montana': 42.7,
    'Nebraska': 48.8,
    'Nevada': 49.9,
    'New Hampshire': 43.8,
    'New Jersey': 52.7,
    'New Mexico': 53.4,
    'New York': 45.4,
    'North Carolina': 59.0,
    'North Dakota': 40.4,
    'Ohio': 50.7,
    'Oklahoma': 59.6,
    'Oregon': 48.4,
    'Pennsylvania': 48.8,
    'Rhode Island': 50.1,
    'South Carolina': 62.4,
    'South Dakota': 45.2,
    'Tennessee': 57.6,
    'Texas': 64.8,
    'Utah': 48.6,
    'Vermont': 42.9,
    'Virginia': 55.1,
    'Washington': 48.3,
    'West Virginia': 51.8,
    'Wisconsin': 43.1,
    'Wyoming': 42.0,
    'American Samoa': 81.2,
    'Puerto Rico': 79.3
}

In [24]:
#weatherbase data - was temporarily defunct
def get_avg_temp(city_name, state_name, state_avg):
    print('{}, {}'.format(city_name, state_name))
    us_url = 'http://www.weatherbase.com/weather/state.php3?c=US&s=&countryname=United-States'
    response = requests.get(us_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    state_url = 'http://www.weatherbase.com' + soup.find('a', href = True, text = state_name)['href']
    response = requests.get(state_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    try:
        city_url = 'http://www.weatherbase.com' + soup.find('a', href = True, text = city_name)['href']
        print(city_url)
    except:
        try:
            city_name = city_name.split(' ')[0]
            city_url = 'http://www.weatherbase.com' + soup.find('a', href = True, text = city_name)['href']
            print(city_url)
        except:
            return state_avg
    
    response = requests.get(city_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    avg_temp = soup.find_all("td", class_="data")[0].text
    
    if(avg_temp == '---'):
        ann_avg_temp = soup.find_all("td", class_="data")[1:13]
        month_avg_temps = []
        for month_avg_temp in ann_avg_temp:
            if(month_avg_temp.text != '---'):
                month_avg_temps.append(float(month_avg_temp.text))
        avg_temp = round(sum(month_avg_temps) / len(month_avg_temps), 1)
    else:
        avg_temp = float(avg_temp)
    return avg_temp

In [25]:
#NOAA weather data
def get_average_temp(city_name, state_name):
    locator = Nominatim(user_agent="akdec00")
    location = locator.geocode("{}, {}".format(city_name, state_name))
    latitude = location.latitude
    longitude = location.longitude
    nws_api_url = "https://api.weather.gov/points/{},{}".format(latitude, longitude)
    response = urlopen(nws_api_url)
    json_data = json.loads(response.read())
    wfo = json_data['properties']['cwa']
    climate_url = 'https://www.weather.gov/wrh/Climate?wfo={}'.format(wfo)
    
    
    form_data = {
        'station': '{}, {}'.format(city_name, state_name),
        'product_select': 'Monthly summarized data',
        "4-digit starting year ('por' for first year of record)": 'por',
        "4-digit ending year ('por' for last year of record)": 'por',
        "ui-widget-content ui-corner-all ui-state-default": 'avgt',
        "ui-widget-content ui-corner-all ui-state-default": 'mean'
    }
    
    response = requests.get(climate_url, data = form_data)
    soup = BeautifulSoup(response.text, 'html.parser')
    #ACIS Query Builder or extract from StnData Network (xhr) response or from table summary (HTML)
    avg_temp = soup.find_all("td", class_="seasonSmry")
    return avg_temp

In [ ]:
us_bridges['avg_temp_f'] = us_bridges.apply(lambda r: get_avg_temp(r.city, r.state_names, state_avg_temps[r.state_names]), axis = 1)

Oregon City, Oregon
http://www.weatherbase.com/weather/weather.php3?s=802953&cityname=Oregon-City-Oregon-United-States-of-America
Pittsburgh, Pennsylvania
http://www.weatherbase.com/weather/weather.php3?s=2527&cityname=Pittsburgh-Pennsylvania-United-States-of-America
Antioch, California
http://www.weatherbase.com/weather/weather.php3?s=722040&cityname=Antioch-California-United-States-of-America
Detroit, Michigan
http://www.weatherbase.com/weather/weather.php3?s=73527&cityname=Detroit-Michigan-United-States-of-America
Ouray County, Colorado
http://www.weatherbase.com/weather/weather.php3?s=302650&cityname=Ouray-Colorado-United-States-of-America
Cape Girardeau, Missouri
http://www.weatherbase.com/weather/weather.php3?s=8337&cityname=Cape-Girardeau-Missouri-United-States-of-America
Conway County, Arkansas
http://www.weatherbase.com/weather/weather.php3?s=695130&cityname=Conway-Arkansas-United-States-of-America
Chicago, Illinois
http://www.weatherbase.com/weather/weather.php3?s=3527&cityna

In [ ]:
us_bridges['avg_temp_c'] = us_bridges['avg_temp_f'].apply(lambda t: round((5/9) * (t - 32), 1))

In [ ]:
us_bridges

In [ ]:
us_bridges.to_csv('../data/bridges_avg_temp.csv')